#  GPT-2 Text-Generating Model
* Here we use tensorflow 1 and not 2
* trained on colab
* for more information, please visit this github https://github.com/minimaxir/gpt-2-simple

In [ ]:
%tensorflow_version 1.x
import tensorflow
print("tensorlow version: ", tensorflow.__version__)
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
# !python3 -m spacy download en_core_web_sm
gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
tensorlow version:  1.15.2
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 377Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.81Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 274Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:15, 6.63Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 298Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.86Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.98Mit/s]


In [ ]:
import numpy as np
import random
import tensorflow
import psycopg2
import re
from datetime import datetime
import os
import pickle
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
# import spacy

INDEX_TO_WORD_FILE = "word_to_index.pickle"
STEPS_BEGINNING_FILE = "steps_beginning.pickle"


# nlp = spacy.load("en_core_web_sm")

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Get the data

In [ ]:
conn = psycopg2.connect(
    host="157.230.24.228",
    database="cookix_db",
    user="cookix_user_db",
    password="f9d6UVP6gxEqueopMCiKdpjC0A5Pi5Ww",
)

cursor = conn.cursor()

cursor.execute("SELECT steps FROM recipes_recipe;")
data = cursor.fetchall()
data = [steps[0] for steps in data]

clean_data = [steps for steps in data if steps.strip() != ""]
print(f"steps recipes count: {len(clean_data)}" )
clean_data = clean_data
clean_data[:2]

steps recipes count: 10303


['To trim artichoke, add the juice from half a lemon to a large bowl of cold water.|Cut off the artichoke s stem; rub cut surface with the other lemon half. Peel off all the leaves.|Cut off top one inch of artichoke. Using a spoon with a serrated edge, scrape out the fibrous choke from the center. Rub the artichoke all over with lemon and drop it into lemon water.|Drain before using.|Cut artichoke hearts into quarters.|Combine artichoke hearts, basil, and garlic in processor.|Add oil and blend until smooth.|Transfer to bowl.|Mix in cheeses. Season artichoke pesto to taste with salt and pepper.|Place pesto in center of platter. Surround with shrimp and serve.',
 'Place yogurt, sour cream, chives, tarragon, garlic, mustard, oil, sugar, salt and pepper in a bowl. Zest lemons over bowl. Juice 1 lemon into bowl.|Whisk until smooth.|Add shrimp, broccoli, arugula and noodles. Toss to coat and serve with remaining lemon, cut into wedges, if desired.|Self']

### text cleaning

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

def preprocess_text(sentences: list):
    
    processed_sen = []
    all_words = []
    
    stop_words = set(stopwords.words('english'))
    
    for sen in sentences:
        
        if not ("Ingredients" in sen or "Save Recipe" in sen or "Print Recipe" in sen):
            
            # replace the pipe "|" that we used for steps separation, by space
            sentence = re.sub(r"(\||:|!|\?|=|\*|\{|\}|\'|\-\-|°|\$|&|''|``|\(|\)|\[|\]|#|%)", ' ', sen.lower())
            sentence = re.sub(r"[0-9]+f|[0-9]+\-(inch)?", ' ', sentence)
            sentence = re.sub(r';', ',', sentence)
            sentence = re.sub(r"\-?>", ',', sentence)

            # replace numbers by [number] (valid for 5, 5555, 55-5555 ....)
            sentence = re.sub(r"\s+[0-9]+((\-|/)[0-9]*)*\s+", ' __NUMBER__ ', sentence)
            sentence = re.sub(r"[0-9]+/?[0-9]*", ' __NUMBER__ ', sentence)
            sentence = re.sub(r'(type_27_data|init_step_by_step_images)', ' ', sentence)
            # remove brackets
            sentence = re.sub(r'\(.+\)', ' ', sentence)
            # remove meaningless comments
            sentence = re.sub(r'Serves.*', ' ', sentence)
            # remove meaningless comments
            sentence = re.sub(r'reprinted.*', ' ', sentence)
            
            sentence = re.sub(r'\.', ' . ', sentence)
            # Removing multiple spaces
            sentence = re.sub(r'\s+', ' ', sentence.strip())
            sen_tokens = word_tokenize(sentence)
            #sen_tokens = [w for w in sen_tokens if not w in stop_words]
            sentence = [lemmatizer.lemmatize(w) for w in sen_tokens]
            [all_words.append(w) for w in sentence]
            sentence = " ".join(sentence)
            processed_sen.append(sentence)    

    return processed_sen, all_words

processed_sen, all_words = preprocess_text(clean_data)

n_words = len(all_words)
unique_words = list(set(all_words))
n_unique_words = len(unique_words)

print('Total Words: %d' % n_words)
print('Unique Words: %d' % n_unique_words)
# Total Words: 1079113
# Unique Words: 11631

Total Words: 1558346
Unique Words: 11708


### copy cleaned text to a file, to be read by the GPT model

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
file = open("recipes_steps.txt", mode='w')
for sen in processed_sen:
  file.write(sen+"\n")

file.close()

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [6]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset="recipes_steps.txt",
              model_name='124M',
              steps=4000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.69s/it]


dataset has 1901284 tokens
Training...
[10 | 27.64] loss=2.70 avg=2.70
[20 | 49.52] loss=2.64 avg=2.67
[30 | 71.89] loss=2.55 avg=2.63
[40 | 94.76] loss=2.15 avg=2.51
[50 | 117.70] loss=2.19 avg=2.44
[60 | 140.31] loss=2.35 avg=2.43
[70 | 162.95] loss=2.29 avg=2.41
[80 | 185.73] loss=2.27 avg=2.39
[90 | 208.48] loss=2.08 avg=2.36
[100 | 231.20] loss=2.13 avg=2.33
[110 | 253.89] loss=2.35 avg=2.33
[120 | 276.60] loss=2.21 avg=2.32
[130 | 299.32] loss=2.01 avg=2.30
[140 | 322.09] loss=1.96 avg=2.27
[150 | 344.84] loss=1.88 avg=2.24
[160 | 367.61] loss=1.82 avg=2.21
[170 | 390.39] loss=2.00 avg=2.20
[180 | 413.14] loss=1.89 avg=2.18
[190 | 435.91] loss=1.82 avg=2.16
[200 | 458.66] loss=1.91 avg=2.15
======== SAMPLE 1 ========
 and salt . put garlic mixture in saucepan , bring to a boil over medium heat , stirring constantly until mixture is very emulsified and beginning to form a thick paste , about __NUMBER__ minute . add remaining cheese , vinegar and remaining __NUMBER__ tablespoon sug

## Load a Trained Model Checkpoint

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="LORD",
              nsamples=5,
              batch_size=5
              )